In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from data_import_ashrae import DataImportAshrae
from aldi_evaluation_metrics import AldiEvaluationMetrics
from mergedeep import merge
from utils import (
    save_variable,
    load_variable,
    get_daily_resolution,
)


In [2]:
meter_type = 0 # TODO: this is hourly, need to do daily
aldi_name = 'vae'

df_all = pd.read_csv("data/outliers/bad_meter_readings_detailed.csv")
df_labels = pd.read_csv("data/pred_discord/discords_vae.csv")
df_all['pred_labels'] = df_labels
df_all = df_all[(df_all["meter"] == meter_type)]
list_site_id = list(range(0,16)) 
list_site_name = [f'Site {i}' for i in list_site_id]

dict_all_pred_labels = {}
dict_all_true_labels = {}

for site_id in list_site_id:
    df_all_site = df_all[df_all['site_id'] == site_id]
    df_true_labels = DataImportAshrae().get_label_data([meter_type], [site_id])
    df_pred_labels = pd.DataFrame(index=df_true_labels.index)
    
    for bdg in df_all_site['building_id'].unique():
        aux_df = df_all_site[df_all_site['building_id'] == bdg]
        aux_df.index = pd.to_datetime(aux_df['timestamp'])
        aux_df = df_pred_labels.join(aux_df)
        df_pred_labels[f'is_discord_{bdg}'] = aux_df['pred_labels'].values

#     print(df_pred_labels)
    
    df_pred_labels = df_pred_labels.fillna(0)
    
    # transform to daily
    df_pred_labels_daily = get_daily_resolution(
        df_hourly_data=df_pred_labels, 
        agg_method='majority_plus')
    df_true_labels_daily = get_daily_resolution(
        df_hourly_data=df_true_labels, 
        agg_method='majority_plus')

    # keep track of all resuts
    dict_all_pred_labels[list_site_name[site_id]] = df_pred_labels_daily
    dict_all_true_labels[list_site_name[site_id]] = df_true_labels_daily

print(AldiEvaluationMetrics().get_roc_auc(
    df_true=pd.concat(dict_all_true_labels, axis=1),
    df_pred=pd.concat(dict_all_pred_labels, axis=1)
    )
)

AldiEvaluationMetrics().get_class_report(
    df_true=pd.concat(dict_all_true_labels, axis=1),
    df_pred=pd.concat(dict_all_pred_labels, axis=1),
    aldi_impl = aldi_name, 
    level_name = 'all', 
    meter_type = meter_type,
    path = ''
)


0.4092986176073531


'              precision    recall  f1-score   support\n\n           0       0.89      0.75      0.82    469166\n           1       0.03      0.06      0.04     47992\n\n    accuracy                           0.69    517158\n   macro avg       0.46      0.41      0.43    517158\nweighted avg       0.81      0.69      0.74    517158\n'

<Figure size 720x720 with 0 Axes>